In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from functools import partial
from multiprocessing import Pool, cpu_count
import time
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch
from optimization.optimization import LinearRegression, train_model, normal_eq_lr
from optimization.datasets import sim_linear_model

## Simulations

The goal of these simulations is to create cases where we have n samples (could be low or high) and m features (1-1000). In the context of an optimization problem, compare the performance between SGD, Newton's method, and Quasi-Newton for each case.

In [2]:
# Functions for saving/loading results
def save_data(n, n_sim, n_epochs):
    file = 'sim_data_'+str(n)+'-obs_'+str(n_sim)+'-sims_'+str(n_epochs)+'-epochs.npz'
    np.savez(file, beta_mse=beta_mse, elapsed_time=elapsed_time,loss_history = loss_history)
    
def load_data(fname):
    data = np.load(fname, allow_pickle=True)
    return data['beta_mse'], data['elapsed_time'], data['loss_history']

# Functions for executing simulations in parallel
def parallel_func(seed, n=None, m=None, n_epochs=None, tol=None, tol_abs=None):
    
    # Generate new data
    X, b, y = sim_linear_model(n, m, seed=seed)

    # Normal equation
    start = time.time()
    b_normal = normal_eq_lr(X, y)[:, 0]
    end = time.time()

    beta_mse_normal = float(((b_normal-b)**2).mean())
    elapsed_time_normal = end-start

    # SGD
    b_hat_sgd, loss_hist_sgd, elapsed_sgd =  train_model(
        X, y, method='sgd', n_epochs=n_epochs, seed=seed, tol=tol, tol_abs=tol_abs)

    beta_mse_sgd = float(((b-b_hat_sgd)**2).mean())

    # Newton's method
    b_hat_newton, loss_hist_newton, elapsed_newton = train_model(
        X, y, method='newton', n_epochs=n_epochs, seed=seed, tol=tol, tol_abs=tol_abs)

    beta_mse_newton = float(((b - b_hat_newton)**2).mean())

    # LBFGS
    b_hat_lbfgs, loss_hist_lbfgs, elapsed_lbfgs = train_model(
        X, y, method='lbfgs', n_epochs=n_epochs, seed=seed, tol=tol, tol_abs=tol_abs)

    beta_mse_lbfgs = float(((b - b_hat_lbfgs)**2).mean())

    results = [
        beta_mse_normal, elapsed_time_normal,
        beta_mse_sgd, loss_hist_sgd, elapsed_sgd,
        beta_mse_newton, loss_hist_newton, elapsed_newton,
        beta_mse_lbfgs, loss_hist_lbfgs, elapsed_lbfgs
    ]

    return results

In [3]:
# Number of simulations
n_sim = 100 

# Number of observations
n_obs = [1000, 5000]

# Number of features
ms = [1, 10, 100, 1000] 

# Number of epochs
n_epochs = 250
tol = 1e-6
seeds = np.arange(n_sim)

for n in n_obs:

    # Track loss, beta error, and elapsed time
    loss_history = {
        'sgd': np.zeros((len(ms), n_sim, n_epochs)),
        'newton': np.zeros((len(ms), n_sim, n_epochs)),
        'lbfgs': np.zeros((len(ms), n_sim, n_epochs))
    }

    beta_mse = {
        'normal': np.zeros((len(ms), n_sim)),
        'sgd': np.zeros((len(ms), n_sim)),
        'newton': np.zeros((len(ms), n_sim)),
        'lbfgs': np.zeros((len(ms), n_sim))
    }

    elapsed_time = {
        'normal': np.zeros((len(ms), n_sim)),
        'sgd': np.zeros((len(ms), n_sim)),
        'newton': np.zeros((len(ms), n_sim)),
        'lbfgs': np.zeros((len(ms), n_sim))
    }

    for i, m in enumerate(ms):
        
        # Process in parallel using all cpu's
        parallel_pfunc = partial(parallel_func, n=n, m=m,n_epochs=n_epochs,
                                 tol=tol)
        with Pool(processes=cpu_count()) as pool:
     
            mapping = pool.imap(parallel_pfunc, seeds)
            results = list(tqdm(mapping, total=len(seeds)))
            
        # Unpack pooled results
        beta_mse['normal'][i] = np.array([r[0] for r in results])
        elapsed_time['normal'][i] = np.array([r[1] for r in results])
        
        beta_mse['sgd'][i] = np.array([r[2] for r in results])
        loss_history['sgd'][i] =  np.array([r[3] for r in results])
        elapsed_time['sgd'][i] = np.array([r[4] for r in results])

        beta_mse['newton'][i] = np.array([r[5] for r in results])
        loss_history['newton'][i] = np.array([r[6] for r in results])
        elapsed_time['newton'][i] = np.array([r[7] for r in results])

        beta_mse['lbfgs'][i] = np.array([r[8] for r in results])
        loss_history['lbfgs'][i] = np.array([r[9] for r in results])
        elapsed_time['lbfgs'][i] = np.array([r[10] for r in results])

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
fname_1000 = '../results/sim_data_1000-obs_100-sims_250-epochs.npz'
fname_5000 = '../results/sim_data_5000-obs_100-sims_250-epochs.npz'

beta_mse_1000, elapsed_time_1000, loss_history_1000 = load_data(fname_1000)
beta_mse_1000 = beta_mse_1000.flatten()[0]
elapsed_time_1000 = elapsed_time_1000.flatten()[0]
loss_history_1000 = loss_history_1000.flatten()[0]

beta_mse_5000, elapsed_time_5000, loss_history_5000 = load_data(fname_5000)
beta_mse_5000 = beta_mse_5000.flatten()[0]
elapsed_time_5000 = elapsed_time_5000.flatten()[0]
loss_history_5000 = loss_history_5000.flatten()[0]

In [ ]:
# Loss starts higher as betas increases, but rate of descent is similar
#   We could quantify rate of convergence as the exponential decay rate of loss

# Another thing we can do is to simply report the iteration where there is the minimum loss 
#(this should be the point of convergence)

for i in range(4):
    plt.figure(i)
    plt.title(f'Loss with {ms[i]} betas')
    plt.plot(np.nanmean(loss_history['sgd'][i](axis=0)), label='SGD')
    #plt.plot(loss_history['newton'][i].mean(axis=0), label='Newton')
    #plt.plot(loss_history['lbfgs'][i].mean(axis=0), label='LBFGS')
    plt.ylabel('Loss (MSE)')
    plt.xlabel('Epoch')
    plt.legend(loc="upper right")

In [ ]:
for i in range(4):
    fig, axes = plt.subplots(figsize=(12, 4), ncols=2)
    
    axes[0].violinplot(elapsed_time['normal'][i])
    axes[0].violinplot(elapsed_time['sgd'][i], positions=[1.5])
    axes[0].violinplot(elapsed_time['newton'][i], positions=[2])
    axes[0].violinplot(elapsed_time['lbfgs'][i], positions=[2.5])
    
    axes[1].violinplot(elapsed_time['normal'][i])
    axes[1].violinplot(elapsed_time['sgd'][i], positions=[1.5])
    
    axes[0].set_xticks([1, 1.5, 2,2.5], ['Normal Eq', 'SGD','Newton', 'LBFGS'])
    axes[1].set_xticks([1, 1.5], ['Normal Eq', 'SGD'])
    
    axes[0].set_title(f'Elapsed Time : {ms[i]} betas')
    axes[1].set_title(f'Normal Eq. vs SGD : {ms[i]} betas')
    
    axes[0].set_ylabel('Time (seconds)')
    axes[1].set_ylabel('Time (seconds)')

In [ ]:
# Plot mse comparison
# helper function to get finite vals in the mse only
# Use this for violin plot visualizations
def get_finite_vals(array):
    ii = np.isfinite(array)
    return(array[ii])


for i in range(4):
    fig, axes = plt.subplots(figsize=(12, 4), ncols=2)
    
    try:
        axes[0].violinplot(get_finite_vals(beta_mse['normal'][i]))
    except:
        pass
    try:
        axes[0].violinplot(get_finite_vals(beta_mse['sgd'][i]), positions=[1.5])
    except:
        pass
    try:
        axes[0].violinplot(get_finite_vals(beta_mse['newton'][i]), positions=[2])
    except:
        pass
    try:
        axes[0].violinplot(get_finite_vals(beta_mse['lbfgs'][i]), positions=[2.5])
    except:
        pass
    
    try:
        axes[1].violinplot(get_finite_vals(beta_mse['normal'][i]))
        axes[1].violinplot(get_finite_vals(beta_mse['sgd'][i]), positions=[1.5])

        axes[0].set_xticks([1, 1.5, 2,2.5], ['Normal Eq', 'SGD','Newton', 'LBFGS'])
        axes[1].set_xticks([1, 1.5], ['Normal Eq', 'SGD'])

        axes[0].set_title(f'MSE: {ms[i]} betas')
        axes[1].set_title(f'Normal Eq. vs SGD : {ms[i]} betas')

        axes[0].set_ylabel('MSE')
        axes[1].set_ylabel('MSE')
    except:
        pass


In [ ]:
for i in range(4):
    fig, axes = plt.subplots(figsize=(12, 4), ncols=2)
    
    axes[0].violinplot(get_finite_vals(beta_mse['normal'][i]))
    axes[0].violinplot(get_finite_vals(beta_mse['lbfgs'][i]), positions=[1.5])
    
    axes[1].violinplot(get_finite_vals(beta_mse['normal'][i]))
    axes[1].violinplot(get_finite_vals(beta_mse['lbfgs'][i]), positions=[1.5])
    
    axes[0].set_xticks([1, 1.5], ['Normal Eq', 'LBFGS'])
    axes[1].set_xticks([1, 1.5], ['Normal Eq', 'LBFGS'])
    
    axes[0].set_title(f'MSE: {ms[i]} betas')
    axes[1].set_title(f'Normal Eq. vs LBFGS : {ms[i]} betas')
    
    axes[0].set_ylabel('MSE')
    axes[1].set_ylabel('MSE')

In [ ]:
for i in range(4):
    fig, axes = plt.subplots(figsize=(12, 4), ncols=2)
    
    axes[0].violinplot(get_finite_vals(beta_mse['sgd'][i]))
    axes[0].violinplot(get_finite_vals(beta_mse['lbfgs'][i]), positions=[1.5])
    
    axes[1].violinplot(get_finite_vals(beta_mse['sgd'][i]))
    axes[1].violinplot(get_finite_vals(beta_mse['lbfgs'][i]), positions=[1.5])
    
    axes[0].set_xticks([1, 1.5], ['SGD', 'LBFGS'])
    axes[1].set_xticks([1, 1.5], ['SGD', 'LBFGS'])
    
    axes[0].set_title(f'MSE: {ms[i]} betas')
    axes[1].set_title(f'SGD vs LBFGS : {ms[i]} betas')
    
    axes[0].set_ylabel('MSE')
    axes[1].set_ylabel('MSE')